In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import geocoder
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
print('Libraries imported.')

Libraries imported.


### Getting Illinois map location from csv file

In [3]:
il_data = pd.read_csv('dataset/illinois.csv')

In [4]:
il_data.head()

,Suburbs,Latitude,Longitude
0,Addison,41.93407,-87.99350
1,Algonquin,42.16439,-88.32686
2,Antioch,42.47898,-88.09638
3,Arlington Heights,42.08729,-87.98044
4,Aurora,41.75710,-88.31613


In [8]:
address = 'Chicago Ridge, Illinois'
#geopy
#User_Agent is an http request header that is sent with each request.
geolocator = Nominatim(user_agent="chicago_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Chicago Ridge, Illinois are 41.7014217, -87.7792196.


#### Mapping Neighborhood

In [9]:
neighbor_loc = il_data.loc[0,'Suburbs']
neighbor_lat = il_data.loc[0, 'Latitude']
neighbor_long = il_data.loc[0, 'Longitude']

In [10]:
# create map of Illinois using latitude and longitude values
map_illinois = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for label, lat, lng in zip(il_data['Suburbs'], il_data['Latitude'], il_data['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_illinois)  
    
map_illinois

In [25]:
CLIENT_ID = 'ZP1LBGI24V2QOLYOS25V4ECL22VWRATVFQZHFFIBFUCX1FX4' #  Foursquare ID
CLIENT_SECRET = '5TJC3YVSBFPHF40TQPIRYSY3IZ2EIR0YVIPO5NLNLJ2Z40IC' #  Foursquare Secret
VERSION = '20201303' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZP1LBGI24V2QOLYOS25V4ECL22VWRATVFQZHFFIBFUCX1FX4
CLIENT_SECRET:5TJC3YVSBFPHF40TQPIRYSY3IZ2EIR0YVIPO5NLNLJ2Z40IC


In [26]:
LIMIT = 100
radius = 500
##using api to get data
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighbor_lat, neighbor_long, radius, LIMIT)


In [27]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZP1LBGI24V2QOLYOS25V4ECL22VWRATVFQZHFFIBFUCX1FX4&client_secret=5TJC3YVSBFPHF40TQPIRYSY3IZ2EIR0YVIPO5NLNLJ2Z40IC&v=20201303&ll=41.93407000000008,-87.99349999999998&radius=500&limit=100'

In [28]:
results = requests.get(url).json()

In [29]:
print(results)

{'meta': {'code': 200, 'requestId': '611196a03ef0a359ec02b221'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': '$-$$$$', 'key': 'price'}, {'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Addison', 'headerFullLocation': 'Addison', 'headerLocationGranularity': 'city', 'totalResults': 22, 'suggestedBounds': {'ne': {'lat': 41.93857000450008, 'lng': -87.98746220440565}, 'sw': {'lat': 41.929569995500074, 'lng': -87.99953779559432}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b44c97ff964a5202cfc25e3', 'name': "Portillo's", 'location': {'address': '100 W Lake St', 'crossStreet': 'btwn Addison Rd & Lincoln Ave', 'lat': 41.93367355289031, 'lng': -87.99147069454193, 'labeledLatLngs': [{'label': 'display', 'lat': 41.93367355289031, 'lng': -87.99147069454193}, {'label': '

# get all nearby venues in Illinois

In [36]:
def get_nearby(names, latitudes, longitudes, radius = 500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        # created the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburbs', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [37]:
# type your answer here
il_venues = get_nearby(names = il_data['Suburbs'], latitudes = il_data['Latitude'], longitudes = il_data['Longitude'])

In [38]:
il_venues

,Suburbs,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,41.93407,-87.99350,Portillo's,41.933674,-87.991471,Hot Dog Joint
1,Addison,41.93407,-87.99350,Lou Malnati's Pizzeria,41.934673,-87.991551,Pizza Place
2,Addison,41.93407,-87.99350,Pyramid Club,41.935948,-87.994557,Bar
3,Addison,41.93407,-87.99350,Uncle Harry's Old Fashioned Ice Cream & Coffee,41.933732,-87.990793,Ice Cream Shop
4,Addison,41.93407,-87.99350,La Magdalena,41.932218,-87.988934,Mexican Restaurant
...,...,...,...,...,...,...,...
2875,Winnetka,42.10581,-87.73367,Tocco,42.105038,-87.734985,Italian Restaurant
2876,Winnetka,42.10581,-87.73367,Love's Yogurt,42.105337,-87.731633,Sandwich Place
2877,Winnetka,42.10581,-87.73367,Bella's Day Spa,42.104902,-87.734817,Spa
2878,Winnetka,42.10581,-87.73367,Winnetka Fitness Center,42.108139,-87.733253,Gym / Fitness Center


### References:<br>
https://developer.foursquare.com/docs/api-reference/venues/explore/  <br>
https://developer.foursquare.com/docs/places-api/versioning/ <br>
https://www.kaggle.com/kristoft/tutorial-foursquare-api-search